In [8]:
import torch
import time
import numpy as np 

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi,dtype=torch.float64)
ZERO = torch.tensor([0.]).to(device)

# Constants
freq = 2
sigma = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Gaussian function and its gradients
def gaussian(x):
    return torch.exp(-torch.sum((x - 0.5)**2, dim=1, keepdim=True) / (2 * sigma**2))

def gaussian_grad_1(x):
    return gaussian(x) * (-(x[:, 0:1] - 0.5) / (sigma**2))

def gaussian_grad_2(x):
    return gaussian(x) * (-(x[:, 1:2] - 0.5) / (sigma**2))

def gaussian_grad_3(x):
    return gaussian(x) * (-(x[:, 2:3] - 0.5) / (sigma**2))

# Exact function u_exact
def u_exact(x):
    return gaussian(x) * torch.cos(2 * pi * freq * x[:, 0:1])

# Manually computed gradients
def u_grad_1(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_1(x) - 2 * pi * freq * torch.sin(2 * pi * freq * x[:, 0:1]) * gaussian(x)

def u_grad_2(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_2(x)

def u_grad_3(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_3(x)

# Collecting the gradients in a list
def u_exact_grad():
    return [u_grad_1, u_grad_2, u_grad_3]

# # Manually computed Laplacian
def laplace_u_exact(x):
    return - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
            + torch.cos(2*pi*freq*x[:,0:1])*( gaussian(x) * ( ((x[:,0:1] - 0.5)/(sigma**2))**2 -1/(sigma**2))  ) \
            -( (2*pi*freq)**2 * torch.cos(2*pi*freq*x[:,0:1]) * gaussian(x) + (2*pi*freq)*torch.sin(2*pi*freq*x[:,0:1]) * gaussian_grad_1(x) ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * (gaussian(x) * ( ((x[:,1:2] - 0.5)/(sigma**2))**2 -1/(sigma**2) )  ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * ( gaussian(x) * ( ((x[:,2:3] - 0.5)/(sigma**2))**2 -1/(sigma**2) )   ) \


# def laplace_u_exact(x):
#     # Second derivative w.r.t x_1
#     term_1 = -(2 * pi * freq)**2 * torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x)
#     term_1 += torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 0:1] - 0.5)**2 / sigma**4 - 1 / sigma**2)
#     term_1 += 2 * pi * freq * torch.sin(2 * pi * freq * x[:, 0:1]) * gaussian_grad_1(x)

#     # Second derivative w.r.t x_2
#     term_2 = torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 1:2] - 0.5)**2 / sigma**4 - 1 / sigma**2)

#     # Second derivative w.r.t x_3
#     term_3 = torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 2:3] - 0.5)**2 / sigma**4 - 1 / sigma**2)

#     return term_1 + term_2 + term_3


# Function to compute gradient using autograd
def compute_autograd_grad(u_func, x):
    x.requires_grad_(True)  # Enable gradient tracking for input x
    u = u_func(x)
    u_grad = torch.autograd.grad(outputs=u, inputs=x,
                                 grad_outputs=torch.ones_like(u),
                                 create_graph=True, retain_graph=True)[0]
    return u_grad

# Function to compute Laplacian using autograd
def compute_autograd_laplace(u_func, x):
    u_grad = compute_autograd_grad(u_func, x)
    
    laplacian = 0
    for i in range(x.shape[1]):
        grad_i = u_grad[:, i:i+1]
        u_grad2_i = torch.autograd.grad(outputs=grad_i, inputs=x,
                                        grad_outputs=torch.ones_like(grad_i),
                                        create_graph=True, retain_graph=True)[0][:, i:i+1]
        laplacian += u_grad2_i
    return laplacian

# Generate sample input points
x = torch.rand(1000, 3).to(device)  # Random 3D points

# Compute the manually computed gradients and Laplacian
s_time = time.time()
u_grad_manual_1 = u_grad_1(x)
u_grad_manual_2 = u_grad_2(x)
u_grad_manual_3 = u_grad_3(x)
laplace_manual = laplace_u_exact(x)
print("manully compute derivative: ",time.time() - s_time)

# Compute the gradients and Laplacian using autograd
s_time = time.time()
u_grad_autograd = compute_autograd_grad(u_exact, x)
laplace_autograd = compute_autograd_laplace(u_exact, x)
print("autograd for derivative: ",time.time() - s_time)
# Compute differences for comparison
grad_diff_1 = torch.abs(u_grad_manual_1 - u_grad_autograd[:, 0:1])
grad_diff_2 = torch.abs(u_grad_manual_2 - u_grad_autograd[:, 1:2])
grad_diff_3 = torch.abs(u_grad_manual_3 - u_grad_autograd[:, 2:3])
laplace_diff = torch.abs(laplace_manual - laplace_autograd)

# Display the results
print("Manual Gradient 1 vs Autograd Gradient 1 Difference:")
print(grad_diff_1.sum())

print("\nManual Gradient 2 vs Autograd Gradient 2 Difference:")
print(grad_diff_2.sum())

print("\nManual Gradient 3 vs Autograd Gradient 3 Difference:")
print(grad_diff_3.sum())

print("\nManual Laplacian vs Autograd Laplacian Difference:")
print(laplace_diff.sum())


manully compute derivative:  0.0016942024230957031
autograd for derivative:  0.002557992935180664
Manual Gradient 1 vs Autograd Gradient 1 Difference:
tensor(2.5175e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Gradient 2 vs Autograd Gradient 2 Difference:
tensor(1.3284e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Gradient 3 vs Autograd Gradient 3 Difference:
tensor(1.5352e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Laplacian vs Autograd Laplacian Difference:
tensor(9.2570e-13, device='cuda:0', grad_fn=<SumBackward0>)
